## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind,Weather Description
0,0,Dikson,RU,73.51,80.55,40.69,88,12,12.48,few clouds
1,1,Mount Gambier,AU,-37.83,140.77,44.60,93,40,8.05,scattered clouds
2,2,Iralaya,HN,15.00,-83.23,84.20,79,40,1.88,scattered clouds
3,3,Hilo,US,19.73,-155.09,73.40,64,90,4.70,overcast clouds
4,4,Punta Arenas,CL,-53.15,-70.92,39.20,85,100,5.82,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter the minimum temperature you would like for your vacation: "))
max_temp = float(input("Enter the maximum temperature you would like for your vacation: "))

Enter the minimum temperature you would like for your vacation: 75
Enter the maximum temperature you would like for your vacation: 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                      (city_data_df["Max Temp"] <= max_temp)]

In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind                   0
Weather Description    0
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind,Weather Description
2,2,Iralaya,HN,15.00,-83.23,84.20,79,40,1.88,scattered clouds
6,6,Avarua,CK,-21.21,-159.78,75.20,88,100,14.99,overcast clouds
7,7,Castro,BR,-24.79,-50.01,76.17,49,25,12.06,scattered clouds
11,11,Keti Bandar,PK,24.14,67.45,82.36,81,49,9.15,scattered clouds
15,15,Buala,SB,-8.14,159.59,81.30,79,100,10.27,light rain


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Iralaya,HN,84.20,scattered clouds,15.00,-83.23,
6,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,
7,Castro,BR,76.17,scattered clouds,-24.79,-50.01,
11,Keti Bandar,PK,82.36,scattered clouds,24.14,67.45,
15,Buala,SB,81.30,light rain,-8.14,159.59,
16,Faya,SA,82.40,few clouds,18.39,42.45,
17,Bethel,US,75.20,clear sky,41.37,-73.41,
20,Muborak,UZ,81.19,clear sky,39.26,65.15,
23,Kununurra,AU,77.00,clear sky,-15.77,128.73,
26,Cabinda,AO,77.00,broken clouds,-5.55,12.20,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
# Import numpy
import numpy as np
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)


In [54]:
# 8a. Create the output File (CSV)
output_data_file = ("Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
max_temp = clean_hotel_df["Max Temp"]
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))